In [1]:
import torch
import time
import matplotlib.pyplot as plt
from mlrg.hmc import HMCSampler
from rgflow import RGLayer, RGPartition

In [125]:
class GaussianModel(torch.nn.Module):
    ''' phi4 model energy
        E = (1/2) sum_<ij> |x_i-x_j|^2 + (r/2) sum_i |x_i|^2

        Parameters:
            r :: real - (initial) value of
            '''
    def __init__(self, r=0.):
        super().__init__()
        self.r = torch.nn.Parameter(torch.tensor(r))

    def extra_repr(self):
        return f'r={self.r.item()}'

    def return_param(self):
        return torch.exp(self.r)

    def clone(self):
        mdl = type(self)()
        mdl.load_state_dict(self.state_dict())
        return mdl

    def forward(self, x):
        energy = 0.
        for axis in range(1, x.dim()):
            dx2 = (x.roll(1,axis) - x).square().sum(1)
            energy = energy + dx2 / 2
        x2 = x.square().sum(1)
        energy = energy + torch.exp(self.r) * x2 / 2
        energy = energy.view(energy.shape[:1]+(-1,)).sum(-1)
        return energy

In [225]:
class LinearRGLearner(torch.nn.Module):
    def __init__(self, uv_model, ir_model, device, base_dist='Normal'):
        super().__init__()
        self.uv_model = uv_model.requires_grad_(False)
        # self.ir_model = uv_model.clone().requires_grad_(True)
        self.ir_model = ir_model.requires_grad_(True)
        self.ir_sampler = HMCSampler(self.ir_model, [1, 1])
        self.base_dist = getattr(torch.distributions, base_dist)(0., 1.)
        self.ir_param = self.ir_model.return_param()
        self.uv_param = self.uv_model.return_param()
        self.mat = torch.nn.Parameter(torch.randn(2, 2, requires_grad=True)).to(device)
        self.inv_K = torch.tensor([[2 + self.uv_param, -2],[-2, 2 + self.uv_param]])
        self.K = torch.inverse(self.inv_K)
        self.G = torch.tensor([[1 / self.ir_param, 0],[0, 1]])


    def get_mat(self):
        return self.mat

    def get_ir_model(self):
        return self.ir_model

    def sample(self, samples, device):
        with torch.no_grad():
            return self.rsample(samples, device)

    def rsample(self, samples, device):
        x_ir = self.ir_sampler.sample(device, samples=samples)
        z = self.base_dist.rsample([samples, 1]).to(device)
        return x_ir, z

    def loss(self, samples, device, **kwargs):
        x_ir = self.ir_sampler.sample(device, samples=samples, **kwargs)
        z = self.base_dist.rsample([samples, 1]).to(device)
        X_ir = torch.cat((x_ir, z), dim=1)
        # X_ir.requires_grad_(True)
        x_uv = torch.matmul(X_ir, torch.transpose(self.mat, 0, 1))
        diff = self.uv_model(x_uv) - 1/2 * torch.log(1 / self.ir_param) - self.ir_model(x_ir) - torch.log(torch.abs(torch.det(self.mat))) + 1 / 2 * torch.log(torch.det(self.K))
        Loss = diff    # original Loss
        Loss = Loss.mean()
        return Loss, self.uv_model(x_uv).mean()

    def exact_loss(self, device):
        M = torch.matmul(self.mat, self.G)
        M = torch.matmul(self.inv_K, M)
        Loss = 1 / 2 * torch.trace(torch.matmul(torch.transpose(self.mat, 0, 1), M)) + 1 / 2 * torch.log(torch.det(self.K)) - 1 / 2 * torch.log(1 / self.ir_param) - torch.log(torch.abs(torch.det(self.mat))) - 1
        return Loss

In [226]:
device = 'cpu'
rgl = LinearRGLearner(GaussianModel(r=1.).to(device), GaussianModel(r=1.).to(device), device)
optimizer = torch.optim.Adam(rgl.parameters(), lr=0.005)
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.995)
print(sum(p.numel() for p in rgl.parameters() if p.requires_grad))
print(rgl.get_mat())

5
Parameter containing:
tensor([[-0.9880,  0.8620],
        [-0.1869, -1.3567]], requires_grad=True)


In [230]:
for _ in range(200):
        optimizer.zero_grad()
        loss, *rest = rgl.loss(1000, device)
        # loss = rgl.exact_loss(device)
        loss.backward(retain_graph=True)
        optimizer.step()
        # scheduler.step()
        # print(f'{loss.item()}')
        print(f'{loss.item()} '+' '.join(f'{r.item()}' for r in rest))

0.47716858983039856 0.9642670154571533
0.5018495321273804 0.979393720626831
0.4955291152000427 0.936644434928894
0.5053295493125916 0.9772124290466309
0.48238471150398254 1.0085561275482178
0.4935527443885803 0.9233084917068481
0.4697396755218506 0.9764220714569092
0.4868219494819641 0.9753586053848267
0.4667348563671112 0.9457579255104065
0.5308179259300232 1.0378681421279907
0.5059708952903748 1.015560507774353
0.5040596723556519 1.019698977470398
0.49876639246940613 1.0003248453140259
0.5193162560462952 0.9988695383071899
0.4925503730773926 1.004390835762024
0.4837201237678528 1.0002065896987915
0.4695059657096863 0.9592027068138123
0.46217477321624756 0.9419547915458679
0.4840201735496521 0.9970679879188538
0.5062101483345032 0.9927817583084106
0.4933454394340515 0.9530473947525024
0.4966079294681549 0.962620735168457
0.4736888110637665 1.0142223834991455
0.4948866069316864 1.0256555080413818
0.4876980483531952 0.9675761461257935
0.5136464834213257 1.0378751754760742
0.452215254306

In [185]:
rgl.ir_model

GaussianModel(r=-0.9999980330467224)

In [145]:
print(rgl.get_mat())

Parameter containing:
tensor([[-0.6631,  0.3182],
        [-0.7455, -0.2374]], requires_grad=True)


In [204]:
loss, term = rgl.loss(10000, device)
print(loss)
print(term)
print(rgl.exact_loss(device))

torch.Size([10000, 2])
tensor(3.6691, grad_fn=<MeanBackward0>)
tensor(4.1658, grad_fn=<MeanBackward0>)
tensor(1.0000, grad_fn=<SubBackward0>)


In [208]:
test = rgl.get_ir_model()
hmc = HMCSampler(test, [1,1])
x = hmc.sample(device, samples=3).detach()
E = test(x)
print(x)
print(x.roll(1,1))

tensor([[-0.4934],
        [-2.3520],
        [-4.9539]])
tensor([[-0.4934],
        [-2.3520],
        [-4.9539]])
